In [21]:
import pandas as pd
import string
from collections import Counter, defaultdict
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize, RegexpTokenizer
nltk.download('punkt')
nltk.download('stopwords')

tokenizer = RegexpTokenizer(r'\w+') 

[nltk_data] Downloading package punkt to /home/u4/zawad1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/u4/zawad1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
def lower_case_elements(arr) -> list:
    return [x.lower() for x in arr]

In [23]:
df = pd.read_csv("final_training_data_v2.csv", index_col=0)
df

article  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [24]:
conflict_df = df[df.conflict == 1]
non_conflict_df = df[df.conflict == 0]
pd.set_option('display.max_colwidth', None)
stopwords = nltk.corpus.stopwords.words('spanish')
sample_texts, non_conflict_sample_texts = conflict_df['article'], non_conflict_df['article']
sample_texts = [sample_text.translate(str.maketrans('', '', string.punctuation)) for sample_text in sample_texts]
non_conflict_sample_texts = [sample_text.translate(str.maketrans('', '', string.punctuation)) for sample_text in non_conflict_sample_texts]

In [25]:
sample_texts, non_conflict_sample_texts, stopwords = lower_case_elements(sample_texts), lower_case_elements(non_conflict_sample_texts), set(lower_case_elements(stopwords))


In [26]:
stopwords

{'a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fue',
 'f

In [27]:
def get_stopword_free_texts(sample_texts) -> dict:
    res_sample_texts = []
    for text in sample_texts:
        text_without_stopwords = []
        tokens = word_tokenize(text)
        
        for token in tokens:
            if token not in stopwords:
                text_without_stopwords.append(token)
                
        if text_without_stopwords:
            txt = " ".join(text_without_stopwords)
            res_sample_texts.append(txt)
        
    return res_sample_texts

sample_texts = get_stopword_free_texts(sample_texts)
non_conflict_sample_texts = get_stopword_free_texts(non_conflict_sample_texts)

In [33]:
def get_tf_idf(text_list):
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(text_list)
    df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    df = df.sort_values('TF-IDF', ascending=False)
    return df

pos_tf_idf = get_tf_idf(sample_texts)
neg_tf_idf = get_tf_idf(non_conflict_sample_texts)

print (neg_tf_idf.head(25))

                  TF-IDF
cárceles        0.338593
prisiones       0.279909
carcelarias     0.268267
disponible      0.229642
pandillas       0.224946
presos          0.217957
criminales      0.141571
carcelaria      0.122289
sistema         0.116514
segregación     0.108104
reclusos        0.098457
prisión         0.095172
pueden          0.091254
cárcel          0.085792
dura            0.085405
redes           0.083521
condiciones     0.078546
estructuras     0.076223
dentro          0.074797
hacinamiento    0.073373
países          0.073245
ibíd            0.072861
preventiva      0.071974
penitenciarios  0.068965
penitenciario   0.067149


In [34]:
pos_tf_idf.to_csv("conflict_tf_idf.csv")
neg_tf_idf.to_csv("non_conflict_tf_idf.csv")
